<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
رمزگشایی از روایت‌های خاموش: نجات دادن جوانان از چنگال خودکشی
</font>
</h1>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مقدمه و صورت مسئله
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در این چالش، شما به عنوان یک دانشمند داده با قلبی مشتاق برای نجات جان‌ها، ماموریت دارید که داده‌های گزارش‌دهی ملی مرگ‌های خشونت‌آمیز را تجزیه و تحلیل کنید. این داده‌ها شامل خلاصه‌ای از گزارش‌های انتظامی و پزشکی قانونی می‌باشد که مربوط به خودکشی‌های انجام‌شده توسط جوانان است. هدف از این تجزیه و تحلیل، شناسایی عوامل کلیدی است که می‌تواند به پیشگیری از خودکشی در آینده کمک کند.
داده‌های ارائه‌شده شامل روایت‌های مختلف و غیرقابل شناسایی است که توسط کارشناسان دولتی تهیه شده‌اند. این داده‌ها خلاصه‌ای از ماجراهای تلخی هستند که باید با دقت و حساسیت بررسی شوند.
</font>
</p>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
وارد کردن کتابخانه‌های مورد نیاز
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    ابتدا کتابخانه‌های مورد نیازتان را وارد کنید.
</font>
</p>

In [14]:
import pandas as pd
import numpy as np
import torch
import nltk 
import matplotlib.pyplot as plt 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معرفی مجموعه داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
مجموعه داده آموزشی شامل ۳۰۰۰ سطر است که در جدول زیر، توضیحات هر ستون آمده است.
</font>
</p>

<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|ستون|توضیحات|
|:------:|:---:|
|uid|شناسه‌ی یکتای هر مورد|
|NarrativeLE|گزارش خلاصه‌ای از اطلاعات مامور قانون|
|NarrativeCME|خلاصه‌ای از اطلاعات گزارش پزشک قانونی/معاینه‌کننده پزشکی|
|DepressedMood|در آن زمان افسرده بوده یا خیر (۰ یا ۱)|
    
</font>
</div>
</center>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    مجموعه داده آزمایش نیز مانند مجموعه آموزش است با این تفاوت که ستون <code>DepressedMood</code> که متغیر هدف مسئله است را در خود ندارد. مجموعه داده آزمایش ۵۰۰ سطر دارد.
</font>
</p>


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
خواندن مجموعه داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    در ابتدا نیاز است فایل‌های مجموعه‌داده را بخوانید. نمونه‌های آموزشی در فایل <code>train.csv</code> و نمونه‌های آزمون که باید دسته‌ی آن‌ها را پیش‌بینی کنید در فایل <code>test.csv</code> ذخیره شده‌اند. اگر لازم دانستید می‌توانید به دلخواه خود بخشی از دادگان آموزشی را به عنوان دادگان اعتبارسنجی نیز جدا کنید.
</font>
</p>

In [3]:
train_data = pd.read_csv('../data/train.csv')
train_data

,uid,NarrativeLE,NarrativeCME,DepressedMood
0,astm,V was a XX XX found unresponsive and hanging i...,V was a XX XX found unresponsive and hanging i...,0
1,eqeq,The V is an XX XX. The V called his girlfriend...,DC NOTES: EVIDENCE OF RECENT ALCOHOL USE; RECE...,1
2,arpf,V is a XX XX who died by suicide via multiple ...,V is a XX XX who died by suicide via multiple ...,0
3,ffal,Police were dispatched for a sick assist/ poss...,V is a XX XX who was found unresponsive in an ...,0
4,dyqf,Victim XX died of unspecified causes with an u...,Victim XX XX died from Self-Inflicted Intraora...,0
...,...,...,...,...
2995,begx,V was XXXX. V was found lying on the bed at hi...,V was XXXX. V was found lying on the bed at hi...,0
2996,coxo,"V was a XX, XX. [DC notes that the V is XX]Per...","V was a XX, XX. [DC notes that the V is XX]Per...",0
2997,cqrl,V was found hanging by a rope in the garage by...,V was a XX XX XX whose manner of death was sui...,0
2998,fftl,Officers responded to the report of a suicide ...,The Victim (V) was a XX XX XX XX. V was disco...,1


In [4]:
test_data = pd.read_csv('../data/test.csv')
test_data

,uid,NarrativeLE,NarrativeCME
0,azjz,V (XX XX) was found by his mother hanging by a...,"The V, a XX, XX, was staying at his father's h..."
1,cwke,V was XXXX. V was found at her residence unres...,V was XXXX. V was found in her residence suffe...
2,eqcg,V was XXXX. V was found at her boyfriend's res...,V was XXXX. V was found in another's residence...
3,abgm,A XX XX (V) was found with a GSW to the head i...,V is a XX XX with a medical history of bipolar...
4,cwwt,LE summary:V Demographics data is consistent w...,"XX, XX, Lived w/. - parents/family COD: Gunsh..."
...,...,...,...
495,abbe,V (XX XX) poisoned himself in his garage. V ...,V (XX XX) poisoned himself in his garage. V ...
496,dxof,Victim XX XX died of a self-intentional gunsho...,Victim XX XX died of a self-intentional gunsho...
497,fhqz,V is a XX XX who died by suicide via acute com...,V is a XX XX who died by suicide via acute com...
498,duwv,"The V was a XX, XX XX XX. The V called 911 at ...",The V was a XX XX. The V's family called LE to...


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌پردازش و مهندسی ویژگی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    در این سوال شما می‌توانید از هر تکنیک پیش‌پردازش/مهندسی ویژگی که در گذشته آموختید، استفاده کنید.
    <br>
    تکنیک‌هایی که استفاده می‌کنید به شکل مستقیم مورد ارزیابی توسط سامانه داوری قرار <b>نمی‌گیرند.</b> بلکه همه آن‌ها در دقت مدل شما تاثیر خواهند گذاشت؛ بنابراین هر چه پیش‌پردازش/مهندسی ویژگی بهتری انجام دهید تا دقت مدل بهبود پیدا کند، امتیاز بیشتری از این سوال کسب خواهید کرد.
</font>
</p>

In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   uid            3000 non-null   object
 1   NarrativeLE    3000 non-null   object
 2   NarrativeCME   3000 non-null   object
 3   DepressedMood  3000 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 93.9+ KB


In [8]:
train_data.isnull().sum()    

uid              0
NarrativeLE      0
NarrativeCME     0
DepressedMood    0
dtype: int64

In [10]:
train_data['NarrativeLE'].value_counts()

NarrativeLE
The V was a XX XX. The injury location was documented as residence-apartment/house. The weapon type was not documented. No circumstances were documented. The manner of death was documented as could not be determined.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [ ]:
def clean_txt(
	text: str = None | None
)-> str:
    text = str(text).lower()
    text = re

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مدل‌سازی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    حال که داده را پاکسازی کرده‌اید، وقت آن است که مدلی آموزش دهید که بتواند متغیر هدف این مسئله را پیش‌بینی کند.
    <br>
    شما مجاز هستید از هر مدلی که آموخته‌اید استفاده کنید. به عبارت بهتر، هدف این سوال پیش‌بینی هرچه بهتر متغیر هدف مسئله است!
</font>
</p>

In [ ]:
# modeling 


<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    معیاری که برای ارزیابی عملکرد مدل انتخاب کرده‌ایم، <code>accuracy</code> نام دارد.
    <br>
    این معیار، سنجه ارزیابی کیفیت مدل شماست. به عبارت بهتر در سامانه داوری هم از همین معیار برای نمره‌دهی استفاده شده است.
    <br>
    پیشنهاد می‌شود با توجه به این معیار، عملکرد مدل خود را برای این ستون <i>DepressedMood </i> جداگانه به دست آورده و آن را اعلام کنید
</font>
</p>

In [ ]:
# evaluate model


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 پیش‌بینی برای داده تست و خروجی
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    پس از مهندسی ویژگی و مدلسازی، الگوریتمی دارید که می‌تواند شما را از متغیرهای مستقل به متغیر هدف برساند.
    <br>
    از این مدل برای پیش‌بینی نمونه‌های موجود در داده تست استفاده کنید و نتایج را در قالب جدول (<code>dataframe</code>) زیر آماده کنید.
</font>
</p>

<div dir=rtl style="direction: rtl;text-align: center;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|ستون|توضیحات|
|------|---|
|DepressedMood|در آن زمان افسرده بوده یا خیر (۰ یا ۱)|
    
</font>
</div>



<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    اسم دیتافریم باید <i>submission</i> باشد؛ در غیر این صورت، سامانه داوری نمی‌تواند تلاش‌ شما را ارزیابی کند.
    <br>
    این دیتافریم تنها شامل یک ستون با اسم <i>DepressedMood</i> است و ۵۰۰ سطر دارد.
    <br>
    به ازای هر سطر موجود در دیتافریم <i>test</i> شما باید یک مقدار پیشبینی شده داشته باشید.
    <br>
    جدول زیر، ۵ سطر ابتدایی دیتافریم <code>submission</code> را نشان می‌دهد. البته در جواب شما، مقادیر ستون <i>DepressedMood</i> ممکن است متفاوت باشد.
</font>
</p>

<div style="text-align: center;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|DepressedMood|
|-----|
|1|
|0|
|0|
|0|
|1|

</font>
</div>



In [ ]:
# predict test samples
submission = pd.DataFrame()
submission


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) تا در صورت نیاز به پشتیبانی امکان بررسی کد شما وجود داشته باشد.
</font>
</p>

In [ ]:
import zipfile

if not os.path.exists(os.path.join(os.getcwd(), 'Suicide.ipynb')):
    %notebook -e Suicide.ipynb

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)
            
submission.to_csv('submission.csv', index=False)
file_names = ['Suicide.ipynb', 'submission.csv']
compress(file_names)